#  Для запуска с ВМ

In [ ]:
import os
import csv
import pandas as pd
from docker_init import run_init

_ = run_init(local=False) # только экспорт конфигов

In [ ]:
% env HADOOP_HOME=/usr/local/hadoop
% env CONF_DIR=/usr/local/hadoop/etc/hadoop
% env WORK_DIR=/home/jovyan/work
% env MAPPER=/home/jovyan/work/mapper.py
% env REDUCER=/home/jovyan/work/reducer.py

In [ ]:
# Загрузить данные в HDFS
! hadoop fs -put spotify/log_mini.csv /tmp/spotify_data.csv

In [ ]:
! hadoop fs -ls /tmp

In [ ]:
! chmod +x $MAPPER
! chmod +x $REDUCER

In [ ]:
! hdfs dfs -rm -r /tmp/spotify_results

# Стрим-вычисление прослушиваний с использованием Hadoop Streaming
! hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.4.jar \
  -input /tmp/spotify_data.csv \
  -output /tmp/spotify_results \
  -mapper $MAPPER \
  -reducer $REDUCER

In [ ]:
!hadoop fs -ls /tmp/spotify_results

In [ ]:
result_csv = f"{os.getenv('WORK_DIR')}/mapreduce_results.csv"

if os.path.exists(result_csv):
    os.remove(result_csv)

In [ ]:
# Создать пустой .csv

with open(result_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([])

In [ ]:
# Добавить названия столбцов
! echo "track_id,count" > $WORK_DIR/mapreduce_results.csv # Добавляем заголовки

In [ ]:
# Загрузить файл с результатами из HDFS
! hadoop fs -cat /tmp/spotify_results/part-00000 | \
  awk '{print $1 "," $2}' | \
  sort -t, -k2 -nr \
  >> $WORK_DIR/mapreduce_results.csv

In [ ]:
df = pd.read_csv(result_csv)
df.head()